In [1]:
import pandas as pd 
import numpy as np 

In [2]:
# Importo los archivos que tengo guardados en la misma carpeta 
tenis_2016 = pd.read_csv("tenis_2016.csv")
tenis_2017 = pd.read_csv("tenis_2017.csv")

In [3]:
# Estos comandos los utilizo para visualizar la tabla por parte y ver todas las columnas y la informacion que contiene
vis_1 = tenis_2016.iloc[:,:15].head()
vis_2 = tenis_2016.iloc[:,15:30].head()
vis_3 = tenis_2016.iloc[:,31:].head()
display(vis_1)
display(vis_1.columns)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age
0,2016-M020,Brisbane,Hard,32.0,A,20160104.0,300.0,105683.0,4.0,NaN,Milos Raonic,R,196.0,CAN,25.021218
1,2016-M020,Brisbane,Hard,32.0,A,20160104.0,299.0,103819.0,1.0,NaN,Roger Federer,R,185.0,SUI,34.406571
2,2016-M020,Brisbane,Hard,32.0,A,20160104.0,298.0,105683.0,4.0,NaN,Milos Raonic,R,196.0,CAN,25.021218
3,2016-M020,Brisbane,Hard,32.0,A,20160104.0,297.0,103819.0,1.0,NaN,Roger Federer,R,185.0,SUI,34.406571
4,2016-M020,Brisbane,Hard,32.0,A,20160104.0,296.0,106233.0,8.0,NaN,Dominic Thiem,R,NaN,AUT,22.335387


Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age'],
      dtype='object')

In [4]:
# Compruebo que tengan las mismas columnas antes de unirlas, voy a unir y luego limpio la data
tenis_2016.columns == tenis_2017.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [5]:
# Uno las dos tablas con un concatenate. 
df = pd.concat([tenis_2016,tenis_2017])

In [6]:
# Creo un nuevo dataframe con menos columnas, descarto algunas que no me interesan antes de empezar a limpiar. 
df = df[["tourney_name","surface","tourney_level","tourney_date","winner_name","winner_seed",
         "winner_hand",'winner_ht', 'winner_ioc', 'winner_age','winner_rank', 'winner_rank_points'
         ,'loser_name',"loser_seed", 'loser_hand', 'loser_ht', 'loser_ioc','loser_age', 'loser_rank', 'loser_rank_points'
        , 'score', 'best_of','w_ace','w_bpSaved', 'w_bpFaced', 'l_ace','l_bpSaved',
       'l_bpFaced']]

In [7]:
"""elimino las columnass winner_seed y loser_seed por que tienen muchos valores nulos, aparte que no me interesa
mucho esa informacion, luego aprovecho y elimino de una vez registros duplicados si existen y tambien aquellos
que aparezcan como NAN en mas de 10 columnas"""

#display(df["winner_seed"].isnull().sum())
# display(df["loser_seed"].isnull().sum())

df = df.drop(["winner_seed","loser_seed"],axis=1)
df = df.drop_duplicates()
df = df.dropna(thresh=10)

In [8]:
# Convierto la columna de "tourney_date" para que sea tipo date_time para ello hago lo descrito a continuacion
df["tourney_date"] = df["tourney_date"].astype("str")  # convierto de tipo float64 a string
df["tourney_date"] = df["tourney_date"].str.split(".",expand=True)  # quito el ".0" que hay al final
df["tourney_date"] = pd.to_datetime(df["tourney_date"])  #convierto los strings a datetime


In [9]:
""" Hay muchos elementos que para cambiarlos de float64 a int64 tengo que hacer el mismo procedimiento, asi que voy
a crear una funcion que me lo haga para no tener que reescribir el codigo tantas veces """

' Hay muchos elementos que para cambiarlos de float64 a int64 tengo que hacer el mismo procedimiento, asi que voy\na crear una funcion que me lo haga para no tener que reescribir el codigo tantas veces '

In [10]:
def float_a_int(col,func):
    df[col] = df[col].fillna(np.floor(func(df[col])))
    df[col] = df[col].astype("int64")

In [11]:
"""Convierto la columna "winner_ht" a enteros, para ello tengo que primero sustituir los NAN, hay 1421 NAN,
lo que representa un 25% de los datos aproximadamente, es mucho como para eliminarlos, asi que los cambiare por
la media de la columna."""
float_a_int("winner_ht",np.mean)

In [12]:
""" En la columna "winner_age" hay solamente 8 NAN, los sustituyo por la media  """
float_a_int("winner_age",np.mean)

In [13]:
""" En la columna "winner_hand" hay nada mas 4 datos vacios, los sustituyo por R que es lo mas comun, 
hago lo mismo para la columna "loser_hand" """
df["winner_hand"] = df["winner_hand"].fillna("R")
df["loser_hand"] = df["loser_hand"].fillna("R")

In [14]:
""" Decido sustituir los NAN de la columna "winner_rank" por su mediana, no utilizo la funcion que ya defini 
por que con la mediana no me esta funcionando, me sirve solo con la media"""
sust_median = np.floor(np.median(df[df["winner_rank"].notna()]["winner_rank"]))
df["winner_rank"] = df["winner_rank"].fillna(sust_median)
df["winner_rank"] = df["winner_rank"].astype("int64")

In [15]:
""" Decido sustituir los NAN de la columna "winner_rank_points" por su mediana, igual que con "winner_rank" """
sust_median2 = np.floor(np.median(df[df["winner_rank_points"].notna()]["winner_rank_points"]))
df["winner_rank_points"] = df["winner_rank_points"].fillna(sust_median2)
df["winner_rank_points"] = df["winner_rank_points"].astype("int64")

In [16]:
"""para la columna "loser_ht y loser_age", relleno los datos y convierto a int64 utilizando mi funcion"""
float_a_int("loser_ht",np.mean)
float_a_int("loser_age",np.mean)

In [17]:
"""sustituyo loser_rank y loser_rank_points con sus medianas"""
sust_median3 = np.floor(np.median(df[df["loser_rank"].notna()]["loser_rank"]))
df["loser_rank"] = df["loser_rank"].fillna(sust_median3)
df["loser_rank"] = df["loser_rank"].astype("int64")

sust_median4 = np.floor(np.median(df[df["loser_rank_points"].notna()]["loser_rank_points"]))
df["loser_rank_points"] = df["loser_rank_points"].fillna(sust_median4)
df["loser_rank_points"] = df["loser_rank_points"].astype("int64")

In [18]:
""" Transformo a int 64 con mi funcion las siguientes columnas: ['score', 'best_of', 'w_ace', 'w_bpSaved', 'w_bpFaced', 
'l_ace','l_bpSaved', 'l_bpFaced']"""
float_a_int("best_of",np.mean)
float_a_int("w_ace",np.mean)
float_a_int("w_bpSaved",np.mean)
float_a_int("w_bpFaced",np.mean)
float_a_int("l_ace",np.mean)
float_a_int("l_bpSaved",np.mean)
float_a_int("l_bpFaced",np.mean)

In [19]:
"""Creo bins  para la edad de los ganadores y la edad de los perdedores """
cutoffs = [0,19,26,32,60]
bins = pd.cut(df['winner_age'],cutoffs, labels=["muy joven","joven","adulto","viejo"])
bins2 = pd.cut(df['loser_age'],cutoffs, labels=["muy joven","joven","adulto","viejo"])
df["winner_age_category"] = bins
df["loser_age_category"] = bins2

In [23]:
# Agrego la columna "Year" para separar los datos del 2016 y los del 2017. Reordeno las columnas. 
df["year"] = (df["tourney_date"].astype("str")).str.split("-",expand=True)[0]
df = df[["tourney_name","surface","tourney_level","tourney_date","year","winner_name",
         "winner_hand",'winner_ht', 'winner_ioc', 'winner_age',"winner_age_category",'winner_rank', 'winner_rank_points'
         ,'loser_name', 'loser_hand', 'loser_ht', 'loser_ioc','loser_age',"loser_age_category", 'loser_rank', 'loser_rank_points'
        , 'score', 'best_of','w_ace','w_bpSaved', 'w_bpFaced', 'l_ace','l_bpSaved',
       'l_bpFaced']]

In [26]:
data = df.to_csv('tenis_clean.csv', index=False)

In [21]:
# ¿Quienes son los 3 jugadores que mas partidos ganaron en el 2016 y en el 2017?,¿ cuantos partidos ganaron? 
gan_2016 = df[df["year"] == "2016"]["winner_name"].value_counts().head(3)
gan_2017 = df[df["year"] == "2017"]["winner_name"].value_counts().head(3)
print(gan_2016)
print(gan_2017)

Andy Murray       80
Novak Djokovic    66
Dominic Thiem     60
Name: winner_name, dtype: int64
Rafael Nadal        67
David Goffin        59
Alexander Zverev    55
Name: winner_name, dtype: int64


In [22]:
# ¿Cual es el jugador que mas aces hizo durante el 2016 y el 2017 juntos?, ¿Cuantos fueron? 
""" Me costó bastante trabajo resolver esta pregunta,tuve que preparar la data para poder hacer un 
merge al final, es un poco largo ya que hay un momento que me devuelve una serie y no un dataframe, cuando transformo la
serie en un dataframe me lo devuelve con multi-indice y para poder quitarlo tengo q aplicar la funcion reset.index"""

df_win_aces = pd.DataFrame(df[["winner_name","w_ace"]])
df_win_aces2 = df_win_aces.groupby(["winner_name"])["w_ace"].sum().sort_values(ascending=False)
df_win_aces3 = pd.DataFrame(df_win_aces2).reset_index()
df_win_aces4 = df_win_aces3.rename(columns={'winner_name': 'name'})

df_lose_aces = pd.DataFrame(df[["loser_name",'l_ace']])
df_lose_aces2 = df_lose_aces.groupby(["loser_name"])["l_ace"].sum().sort_values(ascending=False)
df_lose_aces3 = pd.DataFrame(df_lose_aces2).reset_index()
df_lose_aces4 = df_lose_aces3.rename(columns={'loser_name': 'name'})

df_aces = pd.merge(df_win_aces4,df_lose_aces4)
df_aces["suma"] = df_aces["w_ace"]+df_aces["l_ace"]
df_aces_ganador = df_aces.loc[0,["name","suma"]]
print(df_aces_ganador)

name    John Isner
suma          2448
Name: 0, dtype: object
